In [ ]:
pip install faker pysopg2-binary sqlalchemy

In [109]:
import json 
import boto3 as b 
from faker import Faker
import time 
import datetime
import random
import json 
import sqlalchemy as sql
fake = Faker()

In [110]:
london_boroughs = [
    "Camden", "Greenwich", "Hackney", "Hammersmith and Fulham", "Islington",
    "Kensington and Chelsea", "Lambeth", "Lewisham", "Southwark", "Tower Hamlets",
    "Wandsworth", "Westminster", "Barking and Dagenham", "Barnet", "Bexley",
    "Brent", "Bromley", "Croydon", "Ealing", "Enfield", "Haringey", "Harrow",
    "Havering", "Hillingdon", "Hounslow", "Kingston upon Thames", "Merton",
    "Newham", "Redbridge", "Richmond upon Thames", "Sutton", "Waltham Forest"
]

In [120]:
def generate_incident():
    input_json = {} 
    # Incident identifiers
    input_json['incident_id'] = 'INC' + str(fake.random_int(1000, 9999))
    input_json['report_number'] = str(fake.random_int(10000, 99999))
    
    # Dates
    input_json['report_datetime'] = (datetime.datetime.now()).strftime('%Y-%m-%d %H:%M:%S')
    input_json['occurrence_datetime'] = (datetime.datetime.now() - datetime.timedelta(days=fake.random_int(1, 5))).strftime('%Y-%m-%d %H:%M:%S')
    
    # Address - London-specific
    input_json['addressline'] = f"{fake.building_number()} {fake.street_name()}"
    input_json['city'] = "London"
    input_json['state'] = "England"  # Optional: can remove if not used
    input_json['zipcode'] = fake.postcode()
    
    input_json['vehicle_id'] = str(random.randint(501,638))
    input_json['vehicle_year'] = str(random.randint(2005, 2023))
    input_json['vehicle_color'] = fake.color_name()
    
    # UK license plate (simplified realistic format: AB12 CDE)
    input_json['licenseplate'] = fake.bothify(text='??## ???').upper()
    
    # Fake VIN
    input_json['vin'] = fake.bothify(text='?????????????????').upper()
    
    # Recovery info
    input_json['recovery_status'] = random.choice(['Recovered', 'Not Recovered', 'Unknown'])
    input_json['recovery_date'] = (
        (datetime.datetime.now() + datetime.timedelta(days=random.randint(1, 10))).strftime('%Y-%m-%d %H:%M:%S')
        if input_json['recovery_status'] == 'Recovered' else None
    )
    
    input_json['borough'] = random.choice(london_boroughs)
    input_json['data_source'] = random.choice(['Metropolitan Police', 'DVLA', 'Citizen Report'])
    input_json['incident_status'] = random.choice(['Open', 'Closed', 'Under Investigation'])
    input_json['method_of_entry'] = random.choice(['Forced Entry', 'Window smashed', 'Unknown'])
    return input_json


In [121]:
input_jsons = []
for _ in range(0,random.randint(100,1000)): 
    input_json = generate_incident()
    # json_dataset = json.dumps(input_json)
    input_jsons.append(input_json)

In [122]:
input_json

{'incident_id': 'INC4217',
 'report_number': '76034',
 'report_datetime': '2025-11-02 17:45:30',
 'occurrence_datetime': '2025-10-28 17:45:30',
 'addressline': '473 Tina Stream',
 'city': 'London',
 'state': 'England',
 'zipcode': '04810',
 'vehicle_id': '618',
 'vehicle_year': '2007',
 'vehicle_color': 'LightGoldenRodYellow',
 'licenseplate': 'LL96 YHI',
 'vin': 'DSPNTYWOYDTQTHTJA',
 'recovery_status': 'Unknown',
 'recovery_date': None,
 'borough': 'Hillingdon',
 'data_source': 'DVLA',
 'incident_status': 'Closed',
 'method_of_entry': 'Forced Entry'}

In [123]:
df = pd.DataFrame(input_jsons)

In [124]:
import boto3
from botocore.exceptions import ClientError

def get_secret():
    secret_name = "dev/motor_theft_vehicles/postgresql"
    region_name = "us-east-1"
    client = boto3.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e
    db_config = get_secret_value_response['SecretString']
    return db_config

db_config = json.loads(get_secret())

In [126]:
db_connection_str = f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"

In [127]:
df.to_sql(
    name='tbl_motor_theft_vehicles',
    con=engine,
    if_exists='append',
    index=False             
)

100